In [1]:
!nvidia-smi

Wed Dec 22 06:10:33 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir("/content/gdrive/MyDrive")

Mounted at /content/gdrive


In [3]:
# Install torch dependencies: (use cu110 since colab has CUDA 11)
#!pip install -U torch==1.6.0+cu110 torchvision==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html
!pip install torch==1.7.0 torchvision==0.8.0 torchaudio==0.7.0

# Install mmcv-full thus we could use CUDA operators
#!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu110/torch1.7.0/index.html
!pip install mmcv-full==1.3.8 -f https://download.openmmlab.com/mmcv/dist/cu102/torch1.7.0/index.html

# Install mmdetection
!pip install mmdet
!rm -r mmocr
# Install mmocr
!git clone https://github.com/open-mmlab/mmocr.git
%cd mmocr
!pip install -r requirements.txt
!pip install -v -e .
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
import mmdet
print(mmdet.__version__)
import mmcv
print(mmcv.__version__)
import mmocr
print(mmocr.__version__)
!pip install opencc-python-reimplemented

     |████████████████████████████████| 776.7 MB 4.4 kB/s 
     |████████████████████████████████| 11.8 MB 27.3 MB/s 
     |████████████████████████████████| 7.6 MB 30.7 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-0.11.1+cu111:
      Successfully uninstalled torchvision-0.11.1+cu111
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.10.0+cu111
    Uninstalling torchaudio-0.10.0+cu111:
      Successfully uninstalled torchaudio-0.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.7.0 which is incompatible.
Looking i

In [ ]:
%cd ..
!mkdir mmocr/work_dir
!cp -r mmocr暫存/data mmocr #造字txt
!cp  mmocr暫存/sar_r31_parallel_decoder_chineseocr_20210507-b4be8214.pth mmocr/work_dir #ocr權重
#!cp -r Competitions/test_ctw1500/img mmocr/work_dir
#!cp -r Competitions/test_ctw1500_with_yolo/img mmocr/work_dir
#!cp -r Competitions/test_ctw1500_with_yolo_1219/img mmocr/work_dir
!cp -r Competitions/test_ctw1500_1221/img mmocr/work_dir

#!cp -r Competitions/order_imgs mmocr/work_dir
#!unzip Competitions/test_ctw1500_with_yolo/yolo/order_imgs.zip -d  mmocr/work_dir
!unzip Competitions/test_ctw1500_1221/yolo/order_imgs.zip -d  mmocr/work_dir
%cd mmocr

In [ ]:
!unzip -P 12345 test.zip -d  work_dir

可以直接train的英文ocr

In [ ]:
%cd ..
!cp -r mmocr暫存/11497/imgs mmocr/tests/data/ocr_toy_dataset
!cp -r mmocr暫存/11497/label.txt mmocr/tests/data/ocr_toy_dataset
%cd mmocr

In [ ]:
!python tools/data/utils/txt2lmdb.py -i tests/data/ocr_toy_dataset/label.txt -o tests/data/ocr_toy_dataset/label.lmdb

In [ ]:
from mmocr.datasets import build_dataset
from mmcv import Config
cfg = Config.fromfile('./configs/textrecog/sar/testdata.py')
dataset = build_dataset(cfg.data.train)
!python tools/train.py configs/textrecog/sar/testdata.py --seed 69 --deterministic --work-dir "work_dir/testdata2"

In [ ]:
from mmocr.apis import init_detector, model_inference
img = './tests/data/ocr_toy_dataset/imgs/img_1_髮型工作室_805_1059_613_665.png'
checkpoint = "./work_dir/testdata2/epoch_2.pth"
out_file = 'outputs/1036169.jpg'
config_file = "./work_dir/testdata2/testdata.py"
model = init_detector(config_file, checkpoint, device="cuda:0")
if model.cfg.data.test['type'] == 'MultiRotateAugOCR':
    model.cfg.data.test.pipeline = model.cfg.data.test['datasets'][0].pipeline


result = model_inference(model, img)
print(f'result: {result}')

img = model.show_result(
        img, result, out_file=out_file, show=False)

mmcv.imwrite(img, out_file)

In [ ]:
!wget https://download.openmmlab.com/mmocr/textrecog/sar/dict_printed_chinese_english_digits.txt

In [ ]:
%cd ..
!cp mmocr暫存/11497_epoch_1.pth mmocr/work_dir
%cd mmocr

/content/gdrive/My Drive
/content/gdrive/My Drive/mmocr


中文ocr

In [83]:
!pwd
%rm -r work_dir/outputs1/
%rm -r work_dir/exp/
!mkdir work_dir/exp/
from mmocr.apis import init_detector, model_inference
import os
from opencc import OpenCC
import cv2
from mmocr.datasets import build_dataset
from mmcv import Config
import shutil
import numpy as np

cc = OpenCC('s2t')  # convert from Simplified Chinese to Traditional Chinese

path = os.path.join("work_dir", "img")
print(len(os.listdir(path)))
out_dir = os.path.join("work_dir", "outputs1")
if not os.path.isdir(out_dir):
  os.mkdir(out_dir)
checkpoint = "./work_dir/sar_r31_parallel_decoder_chineseocr_20210507-b4be8214.pth"
config_file = "configs/textrecog/sar/sar_r31_parallel_decoder_chinese.py"  
#config_file = "configs/textrecog/sar/testdata.py"
model = init_detector(config_file, checkpoint, device="cuda:0")
for index, i in enumerate(os.listdir(path)):
  img = os.path.join(path, i)
  image = cv2.imread(img, cv2.IMREAD_GRAYSCALE)
  image = np.expand_dims(image, axis = 2)
  #print(image.shape)
  area = image.shape[0]*image.shape[1]
  out_file = os.path.join(out_dir, i)
  if float(img.split("_")[-1][:-4]) < 0.2 or area < 300:
    converted = "###"
    if model.cfg.data.test['type'] == 'MultiRotateAugOCR':
        model.cfg.data.test.pipeline = model.cfg.data.test['datasets'][0].pipeline
    result = model_inference(model, image)  
    result["text"] = converted  
  else:
    if model.cfg.data.test['type'] == 'MultiRotateAugOCR':
        model.cfg.data.test.pipeline = model.cfg.data.test['datasets'][0].pipeline
    result = model_inference(model, image)
    #print(f'result: {result}')
    #print(result["score"])
    #if result["score"] < 0.55:
    #  shutil.copyfile(os.path.join(path, i), os.path.join("work_dir", "exp", i))
    #  continue
    converted = cc.convert(result["text"])
    #print(converted)
    if "/" in converted: 
      converted = converted.replace("/", "")
    
    result["text"] = converted
  out_file = out_file[:-4] + "_" + str(converted) + ".png"
  if result["score"] < 0.55:
    #print("糟糕")
    img_gray = cv2.imread(os.path.join(path, i), cv2.IMREAD_GRAYSCALE)
    cv2.imwrite(os.path.join("work_dir", "exp", i), img_gray)
    #shutil.copyfile(os.path.join(path, i), os.path.join("work_dir", "exp", i))
    continue
  img = model.show_result(img, result, out_file=out_file, show=False)
  #print(str(index) + "/" + str(len(os.listdir(path))), "out_file", out_file)
  mmcv.imwrite(img, out_file)
print("最後數量：", len(os.listdir(out_dir)))
print("最後數量：", len(os.listdir(os.path.join("work_dir", "exp"))))

/content/gdrive/My Drive/mmocr
7630
Use load_from_local loader
最後數量： 5597
最後數量： 2033


yolo的輸出

In [114]:
!pwd
from mmocr.datasets import build_dataset
from mmcv import Config
from mmocr.apis import init_detector, model_inference
import os
from opencc import OpenCC
import cv2
import numpy as np
cc = OpenCC('s2t')  # convert from Simplified Chinese to Traditional Chinese

path = os.path.join("work_dir", "order_imgs")
print(len(os.listdir(path)))

checkpoint = "./work_dir/sar_r31_parallel_decoder_chineseocr_20210507-b4be8214.pth"
config_file = "configs/textrecog/sar/sar_r31_parallel_decoder_chinese.py"  
model = init_detector(config_file, checkpoint, device="cuda:0")
for index, i in enumerate(os.listdir(path)):
  yolov5_output_dir = os.path.join(path, i)
  ocr_result = ""
  a = os.listdir(yolov5_output_dir)
  a.sort()
  for split_img in a:
    #print(os.path.join(yolov5_output_dir, split_img))
    img = cv2.imread(os.path.join(yolov5_output_dir, split_img))
    #img = np.expand_dims(img, axis = 2)
    img = cv2.resize(img, (256, 256))
    """
    s = []
    t = []
    rotate_array = [90, 0, -90]
    for i in range(3):
      (h, w, d) = img.shape # 讀取圖片大小
      center = (w // 2, h // 2) # 找到圖片中心
      
      # 第一個參數旋轉中心，第二個參數旋轉角度(-順時針/+逆時針)，第三個參數縮放比例
      M = cv2.getRotationMatrix2D(center, rotate_array[i], 1.0)
      
      # 第三個參數變化後的圖片大小
      img = cv2.warpAffine(img, M, (w, h))
      result = model_inference(model, img)
      print(result)
      s.append(result["score"])
      t.append(result["text"])
    print(np.argmax(s))

    converted = cc.convert(t[np.argmax(s)])
    """
    result = model_inference(model, img)
    converted = cc.convert(result["text"])
    if "/" in converted: 
      converted = converted.replace("/", "")
    result["text"] = converted[0]
    ocr_result += result["text"]
  os.rename(yolov5_output_dir, yolov5_output_dir+"_"+ocr_result)
  print(str(index) + "/" + str(len(os.listdir(path))))

/content/gdrive/My Drive/mmocr
701
Use load_from_local loader
0/701
1/701
2/701
3/701
4/701
5/701
6/701
7/701
8/701
9/701
10/701
11/701
12/701
13/701
14/701
15/701
16/701
17/701
18/701
19/701
20/701
21/701
22/701
23/701
24/701
25/701
26/701
27/701
28/701
29/701
30/701
31/701
32/701
33/701
34/701
35/701
36/701
37/701
38/701
39/701
40/701
41/701
42/701
43/701
44/701
45/701
46/701
47/701
48/701
49/701
50/701
51/701
52/701
53/701
54/701
55/701
56/701
57/701
58/701
59/701
60/701
61/701
62/701
63/701
64/701
65/701
66/701
67/701
68/701
69/701
70/701
71/701
72/701
73/701
74/701
75/701
76/701
77/701
78/701
79/701
80/701
81/701
82/701
83/701
84/701
85/701
86/701
87/701
88/701
89/701
90/701
91/701
92/701
93/701
94/701
95/701
96/701
97/701
98/701
99/701
100/701
101/701
102/701
103/701
104/701
105/701
106/701
107/701
108/701
109/701
110/701
111/701
112/701
113/701
114/701
115/701
116/701
117/701
118/701
119/701
120/701
121/701
122/701
123/701
124/701
125/701
126/701
127/701
128/701
129/701
130/701


yolo輸出 + ezocr

In [ ]:
import os
import cv2
import numpy as np
import easyocr
reader = easyocr.Reader(['ch_tra','en'])

path = os.path.join("work_dir", "order_imgs")
print(len(os.listdir(path)))

checkpoint = "./work_dir/sar_r31_parallel_decoder_chineseocr_20210507-b4be8214.pth"
config_file = "configs/textrecog/sar/sar_r31_parallel_decoder_chinese.py"  
model = init_detector(config_file, checkpoint, device="cuda:0")
for index, i in enumerate(os.listdir(path)):
  yolov5_output_dir = os.path.join(path, i)
  ocr_result = ""
  a = os.listdir(yolov5_output_dir)
  a.sort()
  for split_img in a:
    img_path = os.path.join(yolov5_output_dir, split_img)
    bounds = reader.readtext(img_path)
    if len(bounds) != 0:
      if bounds[0][2] < 0.1 or "/" in str(bounds[0][1]):
        print("no")
      else:
      #print(i)  
        #print("str：", bounds[0][1])
        #print("seore：", bounds[0][2])     
        ocr_result+=str(bounds[0][1])        
  if len(ocr_result)>0:
    print(i)
    print(ocr_result)
    os.rename(yolov5_output_dir, yolov5_output_dir+"_"+ocr_result)
    #print(str(index) + "/" + str(len(os.listdir(path))))



"""
for name in a:
  ocr_text = ""
  img_path = os.path.join(path, name)
  print(name)
  bounds = reader.readtext(img_path)
  print(bounds)
  if len(bounds) != 0:
    for i in range(len(bounds)):
      if bounds[i][2] < 0.1 or "/" in str(bounds[i][1]):
        print("no")
      else:
      #print(i)  
        print("str：", bounds[i][1])
        print("seore：", bounds[i][2])     
        ocr_text+=str(bounds[i][1])
    text = name[:-4] +"_"+ ocr_text +".png"
    if len(ocr_text)>0:
      shutil.copyfile(os.path.join(path, name), os.path.join("work_dir", "ezzocr", text)) 
"""

寫入csv

In [ ]:
import csv
import cv2
import re
output_dir = os.path.join("work_dir", "outputs1")
with open('output.csv', 'w',encoding='utf-8') as csvfile:
  a = os.listdir(output_dir)
  a.sort()
  for i in a:
    str_name = i.replace("_", ",")
    str_name_array = str_name.split(",")
    img_name = str_name_array[0]+"_"+str_name_array[1]
    text = str(str_name_array[11])[:-4]
    if text == "":
      text = "###"
    if text == " ":
      text = "###"
    if text == "  ":
      text = "###"
    if "四川省" in text:
      text = text.replace("四川省", "")
    if "<UKN>" in text:
      text = text.replace("<UKN>", "")
    if "供電" in text:
      text = text.replace("供電", "")
    if [str_name_array[2], str_name_array[3]] == [str_name_array[4], str_name_array[5]] or [str_name_array[2], str_name_array[3]] == [str_name_array[6], str_name_array[7]] or [str_name_array[2], str_name_array[3]] == [str_name_array[8], str_name_array[9]]:
      print(os.path.join(output_dir, i))
      continue
    if [str_name_array[4], str_name_array[5]] == [str_name_array[6], str_name_array[7]] or [str_name_array[4], str_name_array[5]] == [str_name_array[8], str_name_array[9]] or [str_name_array[4], str_name_array[5]] == [str_name_array[2], str_name_array[3]]:
      print(os.path.join(output_dir, i))
      continue
    if [str_name_array[6], str_name_array[7]] == [str_name_array[8], str_name_array[9]] or [str_name_array[6], str_name_array[7]] == [str_name_array[2], str_name_array[3]] or [str_name_array[6], str_name_array[7]] == [str_name_array[4], str_name_array[5]]:
      print(os.path.join(output_dir, i))
      continue
    if [str_name_array[8], str_name_array[9]] == [str_name_array[2], str_name_array[3]] or [str_name_array[8], str_name_array[9]] == [str_name_array[4], str_name_array[5]] or [str_name_array[8], str_name_array[9]] == [str_name_array[6], str_name_array[7]]:
      print(os.path.join(output_dir, i))
      continue
    writer = csv.writer(csvfile)     
    remove_nota = u'[’·°–!"#$%&\'()*+,-./:;<=>?@，。?★、…【】（）《》？“”‘’！[\\]^_`{|}~]+'
    print(re.sub(remove_nota, '', text))
    writer.writerow([img_name, str_name_array[2], str_name_array[3], str_name_array[4], str_name_array[5], str_name_array[6],
                  str_name_array[7], str_name_array[8], str_name_array[9], text])



yolov5的結果寫入csv

清除資料夾的字

In [113]:
output_dir = os.path.join("work_dir", "order_imgs")
for i in os.listdir(output_dir):
  str_name = i.replace("_", ",")
  str_name_array = str_name.split(",")
  if len(str_name_array) > 10:
    rename_str = img_name+"_"+str_name_array[2]+"_"+str_name_array[3]+"_"+str_name_array[4]+"_"+str_name_array[5]+"_"+str_name_array[6]+"_"+str_name_array[7]+"_"+str_name_array[8]+"_"+str_name_array[9]+"_"+str_name_array[10]
    os.rename(os.path.join(output_dir, i), os.path.join(output_dir, rename_str))

In [ ]:
import csv
import cv2
output_dir = os.path.join("work_dir", "order_imgs")
with open('output2.csv', 'w',encoding='utf-8') as csvfile:
  a = os.listdir(output_dir)
  a.sort()
  for i in a:
    #print(os.path.join(output_dir, i))
    str_name = i.replace("_", ",")
    str_name_array = str_name.split(",")
    if len(str_name_array) > 11:
      print(i)
      img_name = str_name_array[0]+"_"+str_name_array[1]
      text = str(str_name_array[11])
      if text == "":
        text = "###"
      if text == " ":
        text = "###"
      if text == "  ":
        text = "###"
      if [str_name_array[2], str_name_array[3]] == [str_name_array[4], str_name_array[5]] or [str_name_array[2], str_name_array[3]] == [str_name_array[6], str_name_array[7]] or [str_name_array[2], str_name_array[3]] == [str_name_array[8], str_name_array[9]]:
        print(os.path.join(output_dir, i))
        continue
      if [str_name_array[4], str_name_array[5]] == [str_name_array[6], str_name_array[7]] or [str_name_array[4], str_name_array[5]] == [str_name_array[8], str_name_array[9]] or [str_name_array[4], str_name_array[5]] == [str_name_array[2], str_name_array[3]]:
        print(os.path.join(output_dir, i))
        continue
      if [str_name_array[6], str_name_array[7]] == [str_name_array[8], str_name_array[9]] or [str_name_array[6], str_name_array[7]] == [str_name_array[2], str_name_array[3]] or [str_name_array[6], str_name_array[7]] == [str_name_array[4], str_name_array[5]]:
        print(os.path.join(output_dir, i))
        continue
      if [str_name_array[8], str_name_array[9]] == [str_name_array[2], str_name_array[3]] or [str_name_array[8], str_name_array[9]] == [str_name_array[4], str_name_array[5]] or [str_name_array[8], str_name_array[9]] == [str_name_array[6], str_name_array[7]]:
        print(os.path.join(output_dir, i))
        continue
      writer = csv.writer(csvfile)
      remove_nota = u'[’·°–!"#$%&\'()*+,-./:;<=>?@，。?★、…【】（）《》？“”‘’！[\\]^_`{|}~]+'
      print(re.sub(remove_nota, '', text))
      writer.writerow([img_name, str_name_array[4], str_name_array[5], str_name_array[6], str_name_array[7], str_name_array[8],
                    str_name_array[9], str_name_array[2], str_name_array[3], text])
      rename_str = img_name+"_"+str_name_array[2]+"_"+str_name_array[3]+"_"+str_name_array[4]+"_"+str_name_array[5]+"_"+str_name_array[6]+"_"+str_name_array[7]+"_"+str_name_array[8]+"_"+str_name_array[9]+"_"+str_name_array[10]

      os.rename(os.path.join(output_dir, i), os.path.join(output_dir, rename_str))

In [ ]:
!pip install easyocr
import easyocr
reader = easyocr.Reader(['ch_tra','en'])

In [116]:
!rm -r work_dir/ezzocr
!mkdir work_dir/ezzocr
path = os.path.join("work_dir", "exp")
a = os.listdir(path)
a.sort()
for name in a:
  ocr_text = ""
  img_path = os.path.join(path, name)
  print(name)
  bounds = reader.readtext(img_path)
  print(bounds)
  if len(bounds) != 0:
    for i in range(len(bounds)):
      if bounds[i][2] < 0.1 or "/" in str(bounds[i][1]):
        print("no")
      else:
      #print(i)  
        print("str：", bounds[i][1])
        print("seore：", bounds[i][2])     
        ocr_text+=str(bounds[i][1])
    text = name[:-4] +"_"+ ocr_text +".png"
    if len(ocr_text)>0:
      shutil.copyfile(os.path.join(path, name), os.path.join("work_dir", "ezzocr", text)) 
  

串流輸出內容已截斷至最後 5000 行。
img_20287_537_977_537_1093_488_1093_488_977_0.6371102.png
[([[0, 0], [50, 0], [50, 117], [0, 117]], '醫', 0.031616936044241095)]
no
img_20288_1174_485_1174_654_1083_654_1083_485_0.92766124.png
[([[0, 0], [94, 0], [94, 171], [0, 171]], '首', 0.0013193202579626778)]
no
img_20288_444_969_444_983_339_983_339_969_0.73269325.png
[]
img_20288_512_439_512_629_261_629_261_439_0.98715055.png
[([[0, 5], [241, 5], [241, 191], [0, 191]], '先', 0.9623639357722737)]
str： 先
seore： 0.9623639357722737
img_20288_861_971_860_985_611_983_612_969_0.8590515.png
[]
img_20289_1305_654_1305_664_1253_664_1253_654_0.9165188.png
[]
img_20289_635_886_635_924_619_924_619_886_0.78772384.png
[]
img_20289_773_659_773_810_635_810_635_659_0.9757505.png
[]
img_20289_782_114_823_319_97_465_56_260_0.93413407.png
[([[0, 0], [448, 0], [448, 264], [0, 264]], '明 倫', 0.6331423900420629), ([[551, 9], [730, 9], [730, 285], [551, 285]], '蛋', 0.08073433191955193)]
str： 明 倫
seore： 0.6331423900420629
no
img_20290_632

In [117]:
import csv
import cv2
output_dir = os.path.join("work_dir", "ezzocr")
with open('output3.csv', 'w',encoding='utf-8') as csvfile:
  a = os.listdir(output_dir)
  a.sort()
  for i in a:
    #print(os.path.join(output_dir, i))
    str_name = i.replace("_", ",")
    str_name_array = str_name.split(",")
    img_name = str_name_array[0]+"_"+str_name_array[1]
    text = str(str_name_array[11][:-4])
    text = text.replace(" ", "")
    #print(text)

    if [str_name_array[2], str_name_array[3]] == [str_name_array[4], str_name_array[5]] or [str_name_array[2], str_name_array[3]] == [str_name_array[6], str_name_array[7]] or [str_name_array[2], str_name_array[3]] == [str_name_array[8], str_name_array[9]]:
      print(os.path.join(output_dir, i))
      continue
    if [str_name_array[4], str_name_array[5]] == [str_name_array[6], str_name_array[7]] or [str_name_array[4], str_name_array[5]] == [str_name_array[8], str_name_array[9]] or [str_name_array[4], str_name_array[5]] == [str_name_array[2], str_name_array[3]]:
      print(os.path.join(output_dir, i))
      continue
    if [str_name_array[6], str_name_array[7]] == [str_name_array[8], str_name_array[9]] or [str_name_array[6], str_name_array[7]] == [str_name_array[2], str_name_array[3]] or [str_name_array[6], str_name_array[7]] == [str_name_array[4], str_name_array[5]]:
      print(os.path.join(output_dir, i))
      continue
    if [str_name_array[8], str_name_array[9]] == [str_name_array[2], str_name_array[3]] or [str_name_array[8], str_name_array[9]] == [str_name_array[4], str_name_array[5]] or [str_name_array[8], str_name_array[9]] == [str_name_array[6], str_name_array[7]]:
      print(os.path.join(output_dir, i))
      continue
    writer = csv.writer(csvfile)

    remove_nota = u'[’·°–!"#$%&\'()*+,-./:;<=>?@，。?★、…【】（）《》？“”‘’！[\\]^_`{|}~]+'
    print(re.sub(remove_nota, '', text))
    if text == "":
      print("yes")
      text = "###"
    writer.writerow([img_name, str_name_array[4], str_name_array[5], str_name_array[6], str_name_array[7], str_name_array[8],
                  str_name_array[9], str_name_array[2], str_name_array[3], text])

103新北經字第002919號經紀人郭蘋俵
CGp00
汪見
德
汽機車
魯肉飯
丟
單公教專屬
賣
涌
麵粥
MILK

yes
總清
總幹清潔
事
狌
佩全國漫畫比賽第名
晶
0
升
RD
間
租
耋
星
而
不
耕
專售
專修
oo11
祥
利

么
SpringBed
8
專業熱誠品質服務履約保證0685388
專營店面透天大樓華廈商辦廠房建地工業地農地
鬟
三
絨
露
和
Wo

yes
抬
免
鐘錶
牌GBalim
空
琯
Clole
Cilolle
每
煎蔥太漢
陽
辰L
主治焦慮躁鬱腦神經衰弱等

么職考
4
燒便臘當
佑幼
學
孛頭房價
8
棪
f接
餉
陽
那
釦
鍋燒


蠱
湖
WA
閒
叨抑
燒

yes
筐

yes
格格媒人
媽鳳格
1
Al
邕臻美
直
點里
岫
I
傑鴻
國泰C知音即將滿檔
機車
口HEALTHINSUaAy
基改
薹
b
執強強怕學
國加學進耆
國擊擊蠱生加先中
歡迎訂購
口口松
EBOLA黛寶拉o
文
豊
店舖
飩
踢
耋
理短
班
新
山
D
4徑孽長滑奉套妝圈蛋服
有
鈸
飲
所
陽
價高矣
0朗
山申
室
先
明倫
TNKikehcA
LIHOUKitchen
舖師雞腿
轉學考
言上
日
住
0店
點湯粉餛
允
p
鹹蛋
服
鐘
澄
山1萱
鼎
h
FrJ
技
0
7
和
犛
昱
所
g
鑫
珽
0
魷魚
愛
坊
5
捲簾
勝彩
火
31101佃
GiAipDroop
16
五金百貨
70
茂
釁
腳
挖
預約專線3879966
業髮
咖
7
兒才藝班
RLD
卑
炒
元
邱
躲0
勰

雷
全
匠3
0966086
叩叩
去
黑是
前苗栗新竹地檢署檢察官劉正穆律師主持諮詢專線037325878
語菁英班白然理化班
話
補
習
墨
2035353535353540404040404010101010252525252525251525252525252525

1
0492321314
衣
云1兌
蕭
3
家庭醫
霖
捲
RD
隹
捲
門
仆新窗罩窗程
鋼翮門1門採門工
不不鏽造構速鍋
0918231543
鋁盯
綱綱拂繼門工新窗罩窗程修034777727093666
運動
街
1
互
動
肉肉肉
而
垚

yes
玉
賓
紙璃
彎
Cllinlco

抓
WFAP思薇爾
牌
強
霞
1
威練
預宗
6
遠